<div style="color:#D81F26;
           display:fill;
           border-style: solid;
           border-color:#C1C1C1;
           font-size:14px;
           font-family:Calibri;
           background-color:#373737;">
<h2 style="text-align: center;
           padding: 10px;
           color:#FFFFFF;">
======= Playground Oct 2021 =======
</h2>
</div>

# About this notebook

This notebook is for submission to the Playground of Oct 2021. In addition to the LGBM Classifier which has been used in the notebook.  Continuous work will be done to use other models for the compeition. Some codes are referred and copied from other notebooks in Kaggle.  Detailsl will be given in the Reference below.

## Summary of ML pipeline

* Read the training and testing data sets with memory  optimiztaion 
* Identify categorical and numerical features
* For numerical features, standardize feature's values
* Analyze if there are highly correlated features and automatically remove these features (p.s. no features have been removed by correlation analysis)
* Select the best K featuers which are relevant to the target based on p-value <= 0.01 and score >= 10
* Apply correlation analysis and the best K feature selection for categorical variables 
* Optimize the hyperparameters for LGBM
* Get the optimal hyperparameters for cross-validation and data submission

## Reference

* "TPS - Oct 2021 Model with Memory reduced" by S T MOHAMMED @stmohd (URL: https://www.kaggle.com/stmohd/tps-oct-2021-model-with-memory-reduced)
* "TPS 2021 Oct - LigtGBM Classif - Begginers" by BÁRBARA SULPIS @brbarasulpis  (URL: https://www.kaggle.com/brbarasulpis/tps-2021-oct-ligtgbm-classif-begginers)
* "LightGBM hyperparameter optimisation" by Misha Lisovyi @mlisovyi (https://www.kaggle.com/mlisovyi/lightgbm-hyperparameter-optimisation-lb-0-761?kernelSessionId=5289497)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Additional packages
import category_encoders as encoders
from sklearn.preprocessing import StandardScaler
from pandas.api.types import is_numeric_dtype
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

from sklearn import model_selection, metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import roc_auc_score
import os, psutil

#Lgbm
import lightgbm as lgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform



## Parameter Setting

In [ ]:
p_corr = 0.75
p_feature = 0.01
p_score = 15
# No. of rows to be read from the training data set
p_nrows = 500000

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 50

In [ ]:
def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

# Memory Optimization 

In [ ]:
df_test=pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_train=pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv',nrows=p_nrows)
print("Data imported")

## from: https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

df_train = reduce_memory_usage(df_train, verbose=True)
df_test = reduce_memory_usage(df_test, verbose=True)
print(cpu_stats())
print('Memory reduced')

# Identify categorical and numerical variables

In [ ]:
features=[]
categorical=[]
numerical=[]
for feature in df_train.columns:
    if feature not in ['id', 'target']:
        features.append(feature)
        if df_train.dtypes[feature]=='int8':
            categorical.append(feature)
        if df_train.dtypes[feature]=='float16':
            numerical.append(feature)
        #print(test.dtypes[feature])
print('features obtained')

print ('Size of the base {}'.format(df_train.shape))
print ('Total No. of variables {}'.format (len(categorical) + len(numerical)))
print ('Total No. of categorical variables {}'.format (len(categorical)))
print ('Total No. of numberical variables {}'.format (len(numerical)))


In [ ]:
df_train['target'].value_counts()

## Standaridize the numerical variables

In [ ]:
standardEncoder = StandardScaler()
df_num = pd.DataFrame(standardEncoder.fit_transform(df_train[numerical]), columns =numerical)
print(df_num.shape)


# Correlation analysis for numerical variables

In [ ]:
# Code segment for correlation analysis
def get_corr(df, threshold):

    tmp = df.copy()
    
    print('Before the Correlation analysis = {}'.format(tmp.shape))

    # Create correlation matrix
    corr_matrix = tmp.corr().abs()
#    print(corr_matrix)
    
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print('Highly correlated numerical variables to drop:'.format(to_drop))

    # Drop features 
    tmp.drop(to_drop, axis=1, inplace=True)
    print('After the Correlation analysis = {}'.format(tmp.shape))
    # corr_matrix['Optout_Mobile'].sort_values(ascending=False)
    
    return tmp


In [ ]:
df_num_corr = get_corr(df_num, p_corr)

In [ ]:
del df_num

# Top most relevant features for numerical variables

In [ ]:
def get_feature (training, target,  threshold, t_score):

    fs = SelectKBest(score_func=f_classif, k=len(training.columns))
    # apply feature selection
    X_selected = fs.fit_transform(training, target.values.ravel())
    print('Befoe the SelectKBest = {}'.format(training.shape))

    new_features = [] # The list of features with z p-values

    for i in range(len(training.columns)):
#        print('Feature {}: {:.3f} with p-value {:.3f}'.format(training.columns[i], fs.scores_[i], fs.pvalues_[i]))
        if fs.pvalues_[i] <= threshold and fs.scores_[i] >= t_score:
            new_features.append(training.columns[i])
       
    X_selected_final =  pd.DataFrame(X_selected)
    X_selected_final.columns = training.columns
#    print(X_selected_final.shape)
    X_selected_final = X_selected_final[new_features]
#    print(X_selected_final.shape)

    print('=' * 30)
    print('After the SelectKBest = {}'.format(X_selected_final.shape))

    return X_selected_final


In [ ]:
df_train_num = get_feature(df_num_corr, df_train['target'], p_feature, p_score)

In [ ]:
#df_train_num.hist(figsize=(16,20),color = 'g',xlabelsize=0,ylabelsize=0)

In [ ]:
del df_num_corr

# Correlation analysis for categorical variables

In [ ]:
df_cat_corr = get_corr(df_train[categorical], p_corr)

# Top most relevant features for categorical variables

In [ ]:
df_train_cat = get_feature(df_cat_corr, df_train['target'], p_feature, p_score)
df_train_cat.shape

In [ ]:
#df_train_cat.hist(figsize=(16,20),color = 'g',xlabelsize=0,ylabelsize=0)

In [ ]:
del df_cat_corr

In [ ]:
X = pd.concat([df_train_num,df_train_cat], axis=1)
Y = df_train['target']
print(X.shape)
print('='  *  30)
print(Y.shape)


In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
print('X_train = {}, Y_Train = {}'.format(X_train.shape[0], Y_test.shape[0]))

# LGBM Classification

### Find the hyperparameters

In [ ]:
def learning_rate_010_decay_power_099(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_010_decay_power_0995(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.995, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_005_decay_power_099(current_iter):
    base_learning_rate = 0.05
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,Y_test)],
            'eval_names': ['valid'],
            'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 250,
            'categorical_feature': 'auto'}



param_test ={'num_leaves': sp_randint(100, 1000), 
             'min_child_samples': sp_randint(500, 1000), 
             'min_child_weight': [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.2, scale=0.8),
#             'max_depth' : sp_randint(5, 15),            
             'reg_alpha': [1e-1, 1, 2, 5, 10, 50, 100, 200],
             'reg_lambda': [1e-1, 1, 2, 5, 10, 50, 100, 200]}

#first round of LGBM training
# param_test ={'num_leaves': [50, 55, 60, 65, 70], 
#              'min_child_samples': [400, 425, 450, 475, 500], 
#              'min_child_weight': [0.01, 0.05, 0.1, 0.5, 1],
#              'max_depth' : [6, 8, 10, 12, 14, 16, 18, 20], 
#              'subsample': [0.3, 0], 
#              'colsample_bytree': [0.2,0.4,0.5,0.6,0.8],
#              'reg_alpha': [0.1, 1,  5, 10, 50, 100],
#              'reg_lambda': [0.1, 1, 5, 10, 50, 100]}

In [ ]:
clf = lgb.LGBMClassifier(max_depth=-1, random_state=1234, silent=True, metric='auc', n_estimators=1000,  class_weight='balanced', n_jobs = -1)


In [ ]:
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    #scoring='auc',
    n_jobs = -1,
    cv=3,
    refit=True,
    verbose=500,
    random_state=4563)

In [ ]:
gs.fit(X_train, Y_train, **fit_params)

In [ ]:
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
#opt_parameters = {'colsample_bytree': 0.2461334734170988, 'min_child_samples': 476, 'min_child_weight': 0.1, 'num_leaves': 60, 'reg_alpha': 0.1, 'reg_lambda': 50, 'subsample': 0.3356546639847473} 
opt_parameters = {'objective': 'binary', 'learning_rate': 0.01, 'random_state': 234123
                    , 'colsample_bytree': 0.442550753153399, 'max_depth': -1
                    , 'min_child_samples': 238, 'min_child_weight': 100.0, 'num_leaves': 79
                    , 'reg_alpha': 0.1, 'reg_lambda': 1, 'subsample': 0.7737071420860528} 


#clf_final = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
#clf_final.set_params(**opt_parameters)


#Configure from the HP optimisation
clf_final = lgb.LGBMClassifier(**gs.best_estimator_.get_params())

#Configure locally from hardcoded values
#clf_final = lgb.LGBMClassifier(**clf.get_params())


clf_final = lgb.LGBMClassifier(max_depth=-1, random_state=3453456, silent=True, metric='auc', n_estimators=5000,  class_weight='balanced', n_jobs = -1)

#Train the final model with learning rate decay
#clf_final.fit(X_train, Y_train, **fit_params )  #, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])

### Based on the optimal model hyperparameters, use stratifiedKFold for cross-validation

In [ ]:
preds = np.zeros(df_test.shape[0])

from sklearn.model_selection import KFold,StratifiedKFold

kf = StratifiedKFold(n_splits = 5, random_state=434512,shuffle=True)

auc = []
model_lst = []
n = 0

for train_idx, test_idx in kf.split(X,Y):
    x_train, x_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = Y.iloc[train_idx], Y.iloc[test_idx]
    #model = lgb.LGBMClassifier(**opt_parameters)
    clf_final.fit(x_train, y_train, eval_set = [(x_val,y_val)], early_stopping_rounds = 30, eval_metric = "auc", verbose = 100)
#    preds += model.predict_proba(df_test_sub)[:,1]/kf.n_splits
    model_lst.append(clf_final)
    auc.append(roc_auc_score(y_val, clf_final.predict_proba(x_val)[:, 1]))
#    gc.collect()
    print(f"fold: {n+1}, auc: {auc[n]}")
    n+=1       



In [ ]:
print(model_lst)

# Data Submission

In [ ]:
# Read testing data
df_test_sub = df_test.copy()


print("Testing Data imported")
df_id = df_test_sub.iloc[:,0:1]

# Standardization, correlation and KBest feature selection for numerical variables
df_num_t = pd.DataFrame(standardEncoder.transform(df_test_sub[numerical]), columns =numerical)
df_test_num = df_num_t[df_train_num.columns].copy()
print(df_test_num.shape)

# Correlation and KBest feature selection for categorical variables
df_test_cat = df_test_sub[df_train_cat.columns].copy()
print(df_test_cat.shape)

# Prepare the dataset
df_test_sub.drop(categorical, axis = 1, inplace=True)
df_test_sub.drop(numerical, axis = 1, inplace=True)


df_test_sub = pd.concat([df_test_num, df_test_cat], axis=1)  
print(df_test_sub.shape)

In [ ]:
# Prediction
for i in range(len(model_lst)):
    preds += model_lst[i].predict_proba(df_test_sub)[:,1]/kf.n_splits
df_rst = pd.concat([df_id, pd.DataFrame(preds, columns = ['target'])], axis = 1)
df_rst.to_csv("./submission.csv",index=False)
print('Done!')